In [29]:
import earthaccess
import requests

In [10]:
earthaccess.login()

In [51]:
shortname = "MUR-JPL-L4-GLOB-v4.1"
token_edl = earthaccess.get_edl_token()['access_token']

In [52]:
ummc_search_url = "https://cmr.earthdata.nasa.gov/search/collections.umm_json?token={0}&ShortName={1}"

In [53]:
ummc = requests.get(ummc_search_url.format(token_edl, shortname)).json()

In [57]:
related_urls = ummc["items"][0]["umm"]["RelatedUrls"]
url_descriptions = [e['Description'] for e in related_urls]

In [58]:
vds_desc = "Virtual data set reference file for first decade of MUR record (experimental)"
if vds_desc in url_descriptions:
    
    print("Virtual data set reference file exists")

Virtual data set reference file exists


In [63]:
requests.get(related_urls[url_descriptions.index(vds_desc)]['URL']).json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [64]:
requests.get(related_urls[url_descriptions.index(vds_desc)]['URL'])

<Response [404]>